In [ ]:
import osmnx as ox
import networkx as nx

### Exploring a new library

First thing we do when facing a new problem is looking for the appropriate tools for the job.

We quickly find out about [networkx](https://networkx.org/), a famous Python library for graphs. It is well-documented and used by everyone, boasting almost 15k stars on [GitHub](https://github.com/networkx/networkx).

We could definitely use networkx to model our city network as a graph, and then use one of the many algorithms from graph theory that are already implemented in the library!

By looking around a bit more, though, we stumble upon [osmnx](https://osmnx.readthedocs.io/en/stable/user-reference.html). From its docs:
>OSMnx is a Python package to easily download, model, analyze, and visualize street networks and other geospatial features from OpenStreetMap.

With almost 5k stars on [GitHub](https://github.com/gboeing/osmnx), osmnx looks promising: it includes all the graph stuff from networkx, and it also has geo-spatial functionalities that will allow us to avoid having to build the city graph ourselves!

Let's explore:

In [ ]:
gdf = ox.features_from_place("Varese, Lombardia, Italy", {"building": True})
gdf

The data that osmnx retrieves from openstreetmap is saved in a `GeoDataFrame`, the basic object from [geopandas](https://geopandas.org/en/stable/) used for storing geospatial data.

In [ ]:
type(gdf)

osmnx also has very rich plotting functionalities, built upon the classic plotting library [matplotlib](https://matplotlib.org/):

In [ ]:
fig, ax = ox.plot_footprints(gdf, color="brown", bgcolor="white")

We managed to easily download, store and plot geospatial data about buildings in our target city of Varese. Pretty cool!

Now let's see if we can retrieve the city streets network instead:

In [ ]:
G = ox.graph_from_address(
    address="Piazzale Trento, Varese, Italy",
    dist=2000,
    dist_type="network",
    network_type="drive",
)

From the documentation we understand that we are retrieving a 2000m box centered around an address: this will be the simulated location of the post office warehouse, where the mail is stored and picked up daily by our postman.

We can see that now the downloaded object is a networkx `MultiDiGraph`, i.e. a representation of a Graph which is a directed graph (because streets have a traffic direction) and also a multigraph (because there can be multiple streets linking two intersections):

In [ ]:
type(G)

In [ ]:
node_list = list(G.nodes())

We can also see that nodes are referenced using integer numbers:

In [ ]:
node_list[:10]

Again, the plotting functionality gives us a nice visualization of our network:

In [ ]:
fig, ax = ox.plot_graph(G)

But we can also get back the geospatial information from the graph:

In [ ]:
node_gdf, edge_gdf = ox.graph_to_gdfs(G)

In [ ]:
node_gdf.head(5)

In [ ]:
edge_gdf.head(5)

Now let's select some destinations at random for our deliveries.

We'll assume that all destinations lie at street intersections; this makes the code easier, since we already have them as nodes, but it can be generalised easily with destinations lying on graph edges.

We also use a random seed to ensure reproducibility.

In [ ]:
import random
from random import sample

random.seed(1)
starting_node = list(G.nodes)[0]
destination_nodes = sample(node_list, 8)
delivery_nodes = [starting_node] + destination_nodes

In [ ]:
delivery_nodes

We can use `osmnx` plotting functionalities to highlight the starting point and the delivery points on the map:

In [ ]:
ox.plot.plot_graph(
    G,
    node_color=[
        "blue" if n == starting_node else "red" if n in delivery_nodes else "white"
        for n in G.nodes()
    ],
    node_alpha=[1 if n in delivery_nodes else 0.5 for n in G.nodes()],
    node_size=[
        100 if n == starting_node else 50 if n in delivery_nodes else 20
        for n in G.nodes()
    ],
)

As we mentioned before, the problem of planning multiple deliveries is a famous one and its called **Traveling salesman problem** in the literature, or TSP.

It is a famously hard problem, unsolvable with efficient algorithms, and getting exponentially harder the more destinations need to be visited.

With networkx we have the `traveling_salesman_problem` function that tries to solve the problem approximately, using one of several known algorithms:

In [ ]:
from networkx.algorithms.approximation import traveling_salesman_problem

help(traveling_salesman_problem)

In [ ]:
best_route = traveling_salesman_problem(
    G,
    nodes=delivery_nodes,
    cycle=True,
    method=nx.approximation.simulated_annealing_tsp,
    init_cycle="greedy",
    seed=1,
)

Huh? This is a horrible error!

`NetworkXError: G is not strongly connected`

By inspecting the *stack trace* we see that the code breaks by raising a custom `NetworkXError` after failing the following check:

In [ ]:
nx.is_strongly_connected(G)

As it turns out, the street network of the area we selected is not [**strongly connected**](https://en.wikipedia.org/wiki/Strongly_connected_component#:~:text=A%20directed%20graph%20is%20called,second%20vertex%20to%20the%20first.), meaning that there is at least a pair of nodes that cannot be linked.

This property is necessary for the solvability of a routing problem such as the TSP.

In practical terms, this means that there are two points in the area that cannot be connected by driving from one to the other.

This should be investigated but it's probably an artifact caused by some mapping error or due to some minor driveway; in any case, for our purposes we can "clean" the graph by removing all disconnected components except the main one, which still resembles our original map:

In [ ]:
G_conn = ox.truncate.largest_component(G, strongly=True)
nx.is_strongly_connected(G_conn)

In [ ]:
fig, ax = ox.plot_graph(G_conn)

We should be ready now to try and solve the problem! Let's try again:

In [ ]:
%%time
best_route = traveling_salesman_problem(
    G_conn,
    nodes=delivery_nodes,
    cycle=True,
    method=nx.approximation.simulated_annealing_tsp,
    init_cycle="greedy",
    seed=1,
)

The output format of the solving function is the list of nodes that the salesman (postman) needs to traverse.

We notice that the first one is not the starting point, since there is no way to specify it in the solving function:

In [ ]:
best_route[:5]

But we added the starting point as a destination to visit, so we can reorder the best route to make it start from there:

In [ ]:
def reorder_route(route: list[int], starting_node: int) -> list[int]:
    return (
        route[route.index(starting_node) : -1]
        + route[: route.index(starting_node)]
        + [starting_node]
    )

In [ ]:
best_route_reordered = reorder_route(best_route, starting_node)

In [ ]:
best_route_reordered[:5]

and now we can visualize the best route again using a convenient plotting function:

In [ ]:
ox.plot.plot_graph_route(
    G_conn,
    best_route_reordered,
    route_color="green",
    route_linewidth=4,
    route_alpha=0.5,
    orig_dest_size=4,
    node_color=[
        "blue" if n == starting_node else "red" if n in delivery_nodes else "white"
        for n in G_conn.nodes()
    ],
    node_alpha=[1 if n in delivery_nodes else 0.5 for n in G_conn.nodes()],
    node_size=[
        100 if n == starting_node else 50 if n in delivery_nodes else 20
        for n in G_conn.nodes()
    ],
)

Not bad! We can't know if the solution is the optimal one or even how suboptimal it is, but it looks reasonable enough and not at all obvious.

Well now we have a nice and usable solution, but `osmnx` is really cool and we want to do some more exploration of its features.

For example, by inspecting the data associated to each edge in the graph, we see that by it attached by default some interesting stuff when it downloaded the map:

In [ ]:
G_conn.get_edge_data(*list(G.edges)[0])

If each edge has speed limits (`maxspeed`) and `length` attributes, we could use them to compute the time it takes to traverse each street, and then use this time as a goal for the TSP optimisation.

After all, in real life we don't actually aim at optimising the number of streets our postman has to drive through, but rather the time it takes to deliver all the mail!

And lo and behold, once again there's a function in `osmnx` that does this for us: `add_edge_travel_times`.

In [ ]:
help(ox.routing.add_edge_travel_times)

Following the documentation, we should execute the following two functions to get what we want:

In [ ]:
G_conn = ox.routing.add_edge_speeds(G_conn)
G_conn = ox.routing.add_edge_travel_times(G_conn)

Now `speed_kph` and `travel_time` are added to the edges' data dictionaries...

In [ ]:
G_conn.get_edge_data(*list(G.edges)[0])

... and we can pass the name of the attribute we want to use as weight, i.e. `travel_time`, to the algorithm:

In [ ]:
%%time
best_route_weighted = traveling_salesman_problem(
    G_conn,
    nodes=delivery_nodes,
    weight="travel_time",
    cycle=True,
    method=nx.approximation.simulated_annealing_tsp,
    init_cycle="greedy",
    seed=1,
)

Again, let's reorder the solution and plot it.

In [ ]:
best_route_weighted_reordered = reorder_route(best_route_weighted, starting_node)

In [ ]:
ox.plot.plot_graph_route(
    G_conn,
    best_route_weighted_reordered,
    route_color="orange",
    route_linewidth=4,
    route_alpha=0.5,
    orig_dest_size=4,
    node_color=[
        "blue" if n == starting_node else "red" if n in delivery_nodes else "white"
        for n in G_conn.nodes()
    ],
    node_alpha=[1 if n in delivery_nodes else 0.5 for n in G_conn.nodes()],
    node_size=[
        100 if n == starting_node else 50 if n in delivery_nodes else 20
        for n in G_conn.nodes()
    ],
)

Finally, let's compare the two solutions to see if they are any different:

In [ ]:
ox.plot.plot_graph_routes(
    G_conn,
    [best_route_reordered, best_route_weighted_reordered],
    route_colors=["green", "orange"],
    route_linewidth=4,
    route_alpha=0.5,
    orig_dest_size=4,
    node_color=[
        "blue" if n == starting_node else "red" if n in delivery_nodes else "white"
        for n in G_conn.nodes()
    ],
    node_alpha=[1 if n in delivery_nodes else 0.5 for n in G_conn.nodes()],
    node_size=[
        100 if n == starting_node else 50 if n in delivery_nodes else 20
        for n in G_conn.nodes()
    ],
)

Even though these are approximate solutions and we are not exactly able to evaluate their quality just by looking at them, it's still interesting to be able to pick up some of the differences: for example, the eastern section of the route seems to be longer in the standard "topological" solution compared to the weighted "geographical" one, where travel times favor passing through more central streets.

What would our next steps be if we wanted to get an even better solutions?